<a href="https://colab.research.google.com/github/dijalma/PUC_IA2/blob/master/Aula02_RN_KddCup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregamento da Base

In [2]:
from google.colab import drive # Carrega biblioteca para montar e carregar drive
drive.mount('/content/drive') # Esse código pedirá autenticação

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/My Drive/RN/Exercises2")

!ls
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Aula02_RN_Bank.ipynb	bank.csv	 kddCup.txt
Aula02_RN_KddCup.ipynb	gd_vs_gdm.ipynb


In [4]:
#Carregar a base e visualizar primeiras linhas
df1 = pd.read_csv('kddCup.txt', 
                  sep=' ', 
                  header=0, 
                  index_col=0)  #header=-1 indica que não há linha de header
df1.head()

,protocol_type,flag,src_bytes,logged_in,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,diff_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,out
id,,,,,,,,,,,,,,,,,
1,icmp,SF,1032,0,511,511,0.0,0.0,0.0,0.0,0.00,0.00,1.00,0.0,0.0,0.0,smurf.
2,tcp,SF,204,1,17,17,0.0,0.0,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,normal.
3,tcp,S0,0,0,250,8,1.0,1.0,0.0,0.0,0.06,0.06,0.00,1.0,1.0,0.0,neptune.
4,icmp,SF,520,0,511,511,0.0,0.0,0.0,0.0,0.00,0.00,1.00,0.0,0.0,0.0,smurf.
5,icmp,SF,1032,0,511,511,0.0,0.0,0.0,0.0,0.00,0.00,1.00,0.0,0.0,0.0,smurf.


In [10]:
#dimnesões da base
df1.shape

(48473, 17)

In [11]:
!pip install --upgrade category_encoders

     |████████████████████████████████| 92kB 4.1MB/s 


In [0]:
#dimnesões da base
df1.shape

# Converte atributos categóricos em binário

In [12]:
import category_encoders as ce
encoder = ce.BinaryEncoder()
df_binary = encoder.fit_transform(df1.loc[:,['flag', 'protocol_type']])
df_binary.head()

,flag_0,flag_1,flag_2,flag_3,protocol_type_0,protocol_type_1,protocol_type_2
id,,,,,,,
1,0,0,0,1,0,0,1
2,0,0,0,1,0,1,0
3,0,0,1,0,0,1,0
4,0,0,0,1,0,0,1
5,0,0,0,1,0,0,1


# Divisão em treino e teste

In [13]:
df1 = pd.concat([df_binary, df1.loc[:,'src_bytes':'out']], axis=1)
df1.head()

,flag_0,flag_1,flag_2,flag_3,protocol_type_0,protocol_type_1,protocol_type_2,src_bytes,logged_in,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,diff_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,out
id,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,1,1032,0,511,511,0.0,0.0,0.0,0.0,0.00,0.00,1.00,0.0,0.0,0.0,smurf.
2,0,0,0,1,0,1,0,204,1,17,17,0.0,0.0,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,normal.
3,0,0,1,0,0,1,0,0,0,250,8,1.0,1.0,0.0,0.0,0.06,0.06,0.00,1.0,1.0,0.0,neptune.
4,0,0,0,1,0,0,1,520,0,511,511,0.0,0.0,0.0,0.0,0.00,0.00,1.00,0.0,0.0,0.0,smurf.
5,0,0,0,1,0,0,1,1032,0,511,511,0.0,0.0,0.0,0.0,0.00,0.00,1.00,0.0,0.0,0.0,smurf.


In [0]:
import random
np.random.seed(1) #semente inicial
nlinhas = df1.shape[0]

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df1.loc[:,'flag_0':'dst_host_rerror_rate'], df1.loc[:,'out'], test_size=0.2)

# Normalização

In [0]:
from sklearn import preprocessing
preprocessParams = preprocessing.StandardScaler().fit(x_train)

In [0]:
X_train_normalized = preprocessParams.transform(x_train)
X_test_normalized = preprocessParams.transform(x_test)

# Treinamento da Rede Neural

In [0]:
from keras import Model, Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [30]:
RN = Sequential()
RN.add(Dense(12,input_shape = X_train_normalized.shape[1:],activation = 'sigmoid'))
RN.add(Dense(3,activation = 'sigmoid'))
RN.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 12)                264       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 39        
Total params: 303
Trainable params: 303
Non-trainable params: 0
_________________________________________________________________


In [31]:
#Treino em dummy
onehot = pd.get_dummies(y_train)
target_labels = onehot.columns
y_train = onehot.as_matrix()

#Teste em dummy
onehot = pd.get_dummies(y_test)
target_labels = onehot.columns
y_test = onehot.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [32]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9)
RN.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
trainedRN = RN.fit(X_train_normalized,y_train,
                            epochs = 10, verbose = 1) # poucas épocas de treinamento: 10
score = trainedRN.model.evaluate(X_test_normalized, y_test, verbose = 0)
print('Test score:', score[0])
print('Test accuracy:', score[1])




Epoch 1/10
38778/38778 [==============================] - 2s 45us/step - loss: 0.1641 - acc: 0.6769
Epoch 2/10
38778/38778 [==============================] - 1s 31us/step - loss: 0.0891 - acc: 0.9472
Epoch 3/10
38778/38778 [==============================] - 1s 31us/step - loss: 0.0627 - acc: 0.9784
Epoch 4/10
38778/38778 [==============================] - 1s 32us/step - loss: 0.0488 - acc: 0.9831
Epoch 5/10
38778/38778 [==============================] - 1s 32us/step - loss: 0.0396 - acc: 0.9843
Epoch 6/10
38778/38778 [==============================] - 1s 33us/step - loss: 0.0329 - acc: 0.9857
Epoch 7/10
38778/38778 [==============================] - 1s 32us/step - loss: 0.0280 - acc: 0.9860
Epoch 8/10
38778/38778 [==============================] - 1s 31us/step - loss: 0.0244 - acc: 0.9867
Epoch 9/10
38778/38778 [==============================] - 1s 32us/step - loss: 0.0216 - acc: 0.9870
Epoch 10/10
38778/38778 [==============================] - 1s 32us/step - loss: 0.0194 - acc: 0.9

# Avaliando resultados

In [0]:
# Vamos testar a rede
from sklearn.metrics import confusion_matrix
y_test_predicted = RN.predict(X_test_normalized)
y_test_predicted_indexes = np.argmax(y_test_predicted,axis=1)
y_test_indexes = y_test

In [0]:
#converte para inteiro
import numpy as np
y_test_indexes = [np.where(r==1)[0][0] for r in y_test_indexes]

In [35]:
#matriz de confusão
confMatrix = confusion_matrix(y_test_predicted_indexes, y_test_indexes) 
confMatrix

array([[2134,  101,    0],
       [   0, 1811,    1],
       [   0,   11, 5637]])